In [87]:
%%bash
# setup playground

sh -c "[[ -d ./tmp ]] && { rm -r ./tmp; mkdir ./tmp;} || { mkdir -p ./tmp; }"
# The above oneliner removes the previous ./tmp/ and its content, then creates a new one with the same name
# It is modified after:
# stackoverflow.com/questions/73557323/bash-one-liner-check-if-a-directory-exists-if-so-delete-it-then-create-it-an

touch ./tmp/a{0..1}.txt
touch ./tmp/b{2..3}.txt
echo '*' > ./tmp/.gitignore
tree -a ./tmp

bash: fg: %%bash: no such job
./tmp
├── .gitignore
├── a0.txt
├── a1.txt
├── b2.txt
└── b3.txt

1 directory, 5 files


Brace expansion is used for generating strings. Syntactically, there are two forms: 
1. the sequence expressions
2. the comma-separated strings expression

Here are examples of them respectively:

In [88]:
echo file-{8..12}.txt       # sequence expressions
echo {hello,goodbye}-world  # comma-separated strings expression

file-8.txt file-9.txt file-10.txt file-11.txt file-12.txt
hello-world goodbye-world


As shown above, sequence expressions generate [natural sort order](https://en.wikipedia.org/wiki/Natural_sort_order), while comma-separated string expressions will preserve left to right without altering the order. 

In sequence expression, the range can be integer or alphabet letter, and you can reverse them:

In [89]:
echo {-3..3}
echo {3..-3}
echo {a..z}
echo {z..a}

-3 -2 -1 0 1 2 3
3 2 1 0 -1 -2 -3
a b c d e f g h i j k l m n o p q r s t u v w x y z
z y x w v u t s r q p o n m l k j i h g f e d c b a


However, they can only be either integer or alphabet letter, not mixed. If you mix them the expansion will fail silently:

In [90]:
echo {-3..z}

{-3..z}


When using sequence expression of integer range, if one of the integers is prefixed with 0, the result will be **Zero-Padded Numeric Strings**. (The shell attempts to force all generated terms to contain the same number of digits where necessary.) With Zero-Padded Numeric Strings, the sort order will be **Lexicographical Order**:

In [91]:
echo {07..10}
echo {0010..7}

07 08 09 10
0010 0009 0008 0007


When using sequence expression of alphabet letter as range, You can extend the alphabetical sequence to include different case:

In [92]:
echo {A..z}
echo {z..A}

A B C D E F G H I J K L M N O P Q R S T U V W X Y Z [  ] ^ _ ` a b c d e f g h i j k l m n o p q r s t u v w x y z
z y x w v u t s r q p o n m l k j i h g f e d c b a ` _ ^ ]  [ Z Y X W V U T S R Q P O N M L K J I H G F E D C B A


It seems that the sequence is expanded following the sequence of [ASCII Table](https://www.cs.cmu.edu/~pattis/15-1XX/common/handouts/ascii.html); however, back slash, `\`, that should be in between `[` and `]` is missing.

Note the following results when you reverse the capitalization:

In [93]:
echo {a..Z}
echo {Z..a}

a ` _ ^ ]  [ Z
Z [  ] ^ _ ` a


You should be careful when your intended string generation contains space. For example, if you want to generate `- 1-` and `- 3-`, the following won't work:

In [94]:
echo -{ 1, 3}-

-{ 1, 3}-


The above above does not work because it is equivalent to:

In [95]:
echo '-{' '1,' '3}-'

-{ 1, 3}-


As you can see, the spaces in the statement `echo -{ 1, 3}-` has been intepreted as arugument saparator for the echo command.

 To generate `- 1-` and `- 3-` you can use either one of the following:

In [96]:
echo -{' 1',' 3'}-
echo -{\ 1,\ 3}-

- 1- - 3-
- 1- - 3-


Symbol generation is okay too, but you have to quote or escape them:

In [97]:
echo {'*',"*",\*}
echo {',',",",\,}
echo {'{',"{",\{}
echo {\',"'"}
echo {\",'"'}

* * *
, , ,
{ { {
' '
" "


Otherwise the shell will do further expansion on the result of brance expansion if the symbol has special meaning. For example:

In [100]:
echo ./tmp/{3,*}

./tmp/3 ./tmp/a0.txt ./tmp/a1.txt ./tmp/b2.txt ./tmp/b3.txt


In the above `{3,*}` expands to `3 *` then the glob `*` is further expand to name of files.

---

Combine two or more sequences to create permutation and combination:

In [101]:
echo {A,B,C}{1,2,3}

A1 A2 A3 B1 B2 B3 C1 C2 C3


In [102]:
echo {A..C}{1..3}

A1 A2 A3 B1 B2 B3 C1 C2 C3


Below we generate all the binary numbers for an 4-bit register, in ascending order:

In [103]:
echo {0..1}{0..1}{0..1}{0..1}

0000 0001 0010 0011 0100 0101 0110 0111 1000 1001 1010 1011 1100 1101 1110 1111


---

Brace expansions can be nested: 

In [104]:
echo {{c,b},{z,{1..9},Z}}

c b z 1 2 3 4 5 6 7 8 9 Z


Here is a more complecated example for nesting: 

In [105]:
echo ./tmp/{{b,c}?.txt,ucb/{ex,edit}}

./tmp/b2.txt ./tmp/b3.txt ./tmp/c?.txt ./tmp/ucb/ex ./tmp/ucb/edit


Note that in the above, the part `./tmp/{{b,c}?.txt,...}`, the `?` is expanded to `2` and `3` when it is preceed with `b` because the files exist; however, when preceed with `c` it is left expanded, because the file does not exist.